In [105]:
import pandas as pd
import numpy as np
import gpflow
import matplotlib.pyplot as plt
import pickle

import utils as u
from models import Poisson_Model
%matplotlib inline


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


//anaconda/envs/tensorflow/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [106]:
data = pd.read_csv('../data/gp_weekly_grid_parkslope.csv')
data['DATETIME'] = pd.to_datetime(data['DATETIME'])

data_dict = u.preprocess(data, start_date= '2017/01/01', training_end_date= 26, \
                                                       testing_end_date=12)

e_s = (39.270244500000004 / data.GRID_SQUARE.max())

Training ends on: 2017-07-02 00:00:00
testing ends on: 2017-09-24 00:00:00


In [104]:

m = Poisson_Model(data_dict['X_train'],data_dict['y_train'],use_priors=True, e_s= e_s, period = 52)
m.compile()
opt = gpflow.train.ScipyOptimizer()
print(m.compute_log_likelihood())
opt.minimize(m, maxiter = 1000)


KeyboardInterrupt: 

In [ ]:
pred[1].shape

In [ ]:
plotter = pd.DataFrame()

plotter['pred'] = np.reshape(pred[0], (len(pred[0],)))
plotter['y_test'] = y_test

In [ ]:
plotter.plot()